In [3]:
%cd /mnt/models/mreso/monarch/examples/

/mnt/models/mreso/monarch/examples


[-]E1009 18:33:00.650961 602934 hyperactor/src/mailbox.rs:335] name:undelivered_message_attempt, sender:foo_client[0].client[0], dest:_19k6cbrrjncM[0].comm[0][13147652568889606402], error:broken link: failed to enqueue in MailboxClient, return_handle:foo_client[0].client[0]<hyperactor::mailbox::undeliverable::Undeliverable<hyperactor::mailbox::MessageEnvelope>>


In [ ]:


# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

# @noautodeps
# pyre-ignore-all-errors
import logging
import os
import os
import torch
import torch.distributed as dist
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim

from monarch.tools import commands
from monarch.actor import Actor, current_rank, endpoint
from monarch.actor import Actor, current_rank, endpoint
from monarch.utils import setup_env_for_distributed
from torch.nn.parallel import DistributedDataParallel as DDP
from slurm.utils import get_appdef, get_server_info, create_proc_mesh

os.environ["RUST_BACKTRACE"] = "full"
os.environ["RUST_LOG"] = "debug"
os.environ["SBATCH_RESERVATION"] = "MonarchDDP"
os.environ["TORCH_NCCL_TRACE_BUFFER_SIZE"] = "1000000"


logging.basicConfig(
    level=logging.DEBUG,
    format="%(name)s %(asctime)s %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    force=True,
)


logger: logging.Logger = logging.getLogger(__name__)

class BarrierActor(Actor):
    """This Actor wraps the basic functionality from Torch's DDP example.

    Conveniently, all of the methods we need are already laid out for us,
    so we can just wrap them in the usual Actor endpoint semantic with some
    light modifications.

    Adapted from: https://docs.pytorch.org/tutorials/intermediate/ddp_tutorial.html#basic-use-case
    """

    def __init__(self):
        os.environ["NCCL_DEBUG"] = "INFO"
        self.rank = current_rank().rank

    def _rprint(self, msg):
        """Helper method to print with rank information."""
        print(f"{self.rank=} {msg}")

    @endpoint
    async def setup(self):
        """Initialize the PyTorch distributed process group."""
        self._rprint("Initializing torch distributed")

        WORLD_SIZE = int(os.environ["WORLD_SIZE"])
        LOCAL_RANK = int(os.environ["LOCAL_RANK"])
        
        # initialize the process group
        dist.init_process_group("nccl", rank=self.rank, world_size=WORLD_SIZE, device_id=LOCAL_RANK)
        self._rprint("Finished initializing torch distributed")

    @endpoint
    async def cleanup(self):
        """Clean up the PyTorch distributed process group."""
        self._rprint("Cleaning up torch distributed")
        dist.destroy_process_group()

    @endpoint
    async def demo_basic(self):
        LOCAL_RANK = int(os.environ["LOCAL_RANK"])
        """Run a basic DDP training example."""
        self._rprint(f"{os.environ['NCCL_DEBUG']=}")
        torch.cuda.set_device(LOCAL_RANK)
        self._rprint("Running basic DDP example")
        self._rprint(f"{torch.cuda.device_count()=}")
        self._rprint(f"{torch.cuda.current_device()=}")
        self._rprint(f"{torch.cuda.get_device_name(0)=}")
        self._rprint(f"{torch.cuda.is_initialized()=}")
        t = current_rank().rank * torch.ones(1).to(f'cuda:{LOCAL_RANK}')
        torch.distributed.all_reduce(t)
        self._rprint(f"{t=}")
        self._rprint("Finished running basic DDP example")


async def main():
    num_hosts = 1
    appdef = await get_appdef(num_hosts)
    server_info = await get_server_info(appdef)

    try:
        print("CREATE PROC MESH")
        proc_mesh = await create_proc_mesh(num_hosts, appdef, server_info)
        
        await proc_mesh.logging_option(
            stream_to_client=True,
            aggregate_window_sec=None,
        )

        print("SPAWN ACTORS")
        barrier_actor = proc_mesh.spawn("barrier_actor", BarrierActor)
        print("SETUP ENV")
        await setup_env_for_distributed(proc_mesh)
        print("SETUP CALL")
        await barrier_actor.setup.call()
        print("BASIC DEMO CALL")
        await barrier_actor.demo_basic.call()
        print("CLEAUP CALL")
        await barrier_actor.cleanup.call()

        print("DDP example completed successfully!")

    finally:
        commands.kill(f"slurm:///{server_info.name}")


if __name__ == "__main__":
    await main()

<string>:8: FutureWarning: Setting `workspace='/mnt/models/mreso/monarch/examples'` is deprecated. Use `workspace=monarch.tools.config.workspace.Workspace(dirs=['/mnt/models/mreso/monarch/examples'])` instead.
torchx.schedulers.slurm_scheduler 2025-10-09 18:33:03 INFO unable to get job info for `monarch-root` with `squeue` (squeue: error: Invalid job id: monarch-root
), trying `sacct`
torchx.schedulers.slurm_scheduler 2025-10-09 18:33:03 INFO unable to get job info for `monarch-root` with `sacct` (sacct: fatal: Bad job/step specified: monarch-root
)
monarch.tools.commands 2025-10-09 18:33:03 INFO no existing RUNNING server `slurm:///monarch-root` creating new one...
torchx.runner.api 2025-10-09 18:33:03 INFO Tracker configurations: {}
torchx.runner.api 2025-10-09 18:33:03 INFO Checking for changes in workspace `/root/.monarch/out/tmpj8uye96x/workspace`...
torchx.runner.api 2025-10-09 18:33:03 INFO To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.
t

monarch.tools.commands 2025-10-09 18:33:03 INFO created new `slurm:///1728` waiting for it to be ready...


app_id='monarch-root'
slurm_24_11_0=(24, 11)
slurm_version=(21, 8)
slurm_24_11_0=(24, 11)
slurm_version=(21, 8)
script='#!/bin/bash\n#\n# Generated by TorchX 0.8.0dev0\n# Run with: sbatch --parsable /tmp/tmpz6ih4a4h/torchx-sbatch.sh\n#\n#SBATCH --job-name=mesh0-0 --requeue --ntasks-per-node=1 --cpus-per-task=48 --mem=186777 --gpus-per-node=4\n#SBATCH hetjob\n#SBATCH --job-name=mesh0-1 --requeue --ntasks-per-node=1 --cpus-per-task=48 --mem=186777 --gpus-per-node=4\n\nset -evx\n\nexport PYTHONUNBUFFERED=1\nexport SLURM_UNBUFFEREDIO=1\nexport TORCHX_MAX_RETRIES=0\n\nset +e\nsrun --output=slurm-"$SLURM_JOB_ID"-mesh0-0.out --error=slurm-"$SLURM_JOB_ID"-mesh0-0.err --wait=60 --kill-on-bad-exit=1 --export=ALL,WORKSPACE_DIR=monarch_default_workspace:latest/workspace,PYTHONPATH=monarch_default_workspace:latest/workspace/,CONDA_DIR=monarch_default_workspace:latest/conda,TORCHX_JOB_ID=slurm:///"$SLURM_JOB_ID",TORCHX_INTERNAL_SESSION_ID=031171e6-999e-4ee1-87a3-35db29fe5395 process_allocator --port

slurm.utils 2025-10-09 18:33:18 INFO 
===== Server Info =====
{
  "name": "1728",
  "server_handle": "slurm:///1728",
  "state": "RUNNING",
  "meshes": {
    "mesh0": {
      "host_type": "__UNSET__",
      "hosts": 4,
      "gpus": -1,
      "hostnames": [
        "chi2605",
        "chi2610"
      ]
    }
  }
}
asyncio 2025-10-09 18:33:18 DEBUG Using selector: EpollSelector
monarch._src.actor.allocator 2025-10-09 18:33:18 INFO no match label `procmesh.monarch.meta.com/name` specified in alloc constraints
monarch._src.actor.allocator 2025-10-09 18:33:18 INFO found a single proc mesh `mesh0` in slurm:///1728, will allocate on it
monarch.tools.network 2025-10-09 18:33:18 INFO no AF_INET6 address that can bind TCP sockets for `chi2605:26600` (error: [Errno -3] Temporary failure in name resolution)
monarch.tools.network 2025-10-09 18:33:18 INFO resolved AF_INET address `45.63.75.210:26600` for `chi2605:26600`
monarch.tools.network 2025-10-09 18:33:18 INFO no AF_INET6 address that can bind

app_id='1728'
output='{\n  "meta": {\n    "plugin": {\n      "type": "openapi\\/v0.0.37",\n      "name": "Slurm OpenAPI v0.0.37"\n    },\n    "Slurm": {\n      "version": {\n        "major": 21,\n        "micro": 5,\n        "minor": 8\n      },\n      "release": "21.08.5"\n    }\n  },\n  "errors": [\n  ],\n  "jobs": [\n    {\n      "account": "",\n      "accrue_time": 1760034487,\n      "admin_comment": "",\n      "array_job_id": 0,\n      "array_task_id": null,\n      "array_max_tasks": 0,\n      "array_task_string": "",\n      "association_id": 0,\n      "batch_features": "",\n      "batch_flag": true,\n      "batch_host": "chi2605",\n      "flags": [\n        "TRES_STR_CALC",\n        "JOB_CPUS_SET ",\n        "JOB_MEM_SET"\n      ],\n      "burst_buffer": "",\n      "burst_buffer_state": "",\n      "cluster": "vultr-mi325x-torch",\n      "cluster_features": "",\n      "command": "\\/tmp\\/tmpg0vvyyzz\\/torchx-sbatch.sh",\n      "comment": "",\n      "contiguous": false,\n      "co

...

[chi2610 1046543] [rank5]:[E1009 18:33:43.221512430 ProcessGroupNCCL.cpp:606] [Rank 5] Collective WorkNCCL(SeqNum=1, OpType=ALLREDUCE, NumelIn=1, NumelOut=1, Timeout(ms)=600000) raised the following async exception: NCCL error: remote process exited or there was a network error, NCCL version 2.22.3
[chi2610 1046543] ncclRemoteError: A call failed possibly due to a network error or a remote process exiting prematurely.
[chi2610 1046543] Last error:
[chi2610 1046543] NET/IB : bnxt_re1:1 Got async event : local work queue catastrophic error
[chi2610 1046543] Exception raised from checkForNCCLErrorsInternal at /pytorch/torch/csrc/distributed/c10d/ProcessGroupNCCL.cpp:2707 (most recent call first):
[chi2610 1046543] frame #0: c10::Error::Error(c10::SourceLocation, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >) + 0x88 (0x7fb17d5a0188 in /mnt/models/mreso/monarch/miniforge3/envs/monarch/lib/python3.12/site-packages/torch/lib/libc10.so)
[chi2610 1046543] frame

app_id='1728'
output='{\n  "meta": {\n    "plugin": {\n      "type": "openapi\\/v0.0.37",\n      "name": "Slurm OpenAPI v0.0.37"\n    },\n    "Slurm": {\n      "version": {\n        "major": 21,\n        "micro": 5,\n        "minor": 8\n      },\n      "release": "21.08.5"\n    }\n  },\n  "errors": [\n  ],\n  "jobs": [\n    {\n      "account": "",\n      "accrue_time": 1760034487,\n      "admin_comment": "",\n      "array_job_id": 0,\n      "array_task_id": null,\n      "array_max_tasks": 0,\n      "array_task_string": "",\n      "association_id": 0,\n      "batch_features": "",\n      "batch_flag": true,\n      "batch_host": "chi2605",\n      "flags": [\n        "TRES_STR_CALC",\n        "JOB_CPUS_SET ",\n        "JOB_MEM_SET"\n      ],\n      "burst_buffer": "",\n      "burst_buffer_state": "",\n      "cluster": "vultr-mi325x-torch",\n      "cluster_features": "",\n      "command": "\\/tmp\\/tmpg0vvyyzz\\/torchx-sbatch.sh",\n      "comment": "",\n      "contiguous": false,\n      "co

SupervisionError: Actor ActorId(Ranked(WorldId("_1FxVJVmDQBBm"), 1), "any", 0) exited because of the following reason: failed: proc _1FxVJVmDQBBm[1] is stopped

[-]E1009 18:34:44.767711 602934 hyperactor/src/mailbox.rs:335] name:undelivered_message_attempt, sender:foo_client[0].client[0], dest:_19k6cbrrjncM[0].comm[0][13147652568889606402], error:broken link: failed to enqueue in MailboxClient, return_handle:foo_client[0].client[0]<hyperactor::mailbox::undeliverable::Undeliverable<hyperactor::mailbox::MessageEnvelope>>


[-]E1009 18:35:18.525504 602934 hyperactor/src/channel/net.rs:875] error_msg:session tcp:45.63.75.210:36449.17579174550146264198: failed to deliver message within timeout
[-]E1009 18:35:19.550170 602934 hyperactor/src/channel/net.rs:888] error_msg:session tcp:45.63.75.210:26600.10475225640412375959: failed to receive ack within timeout 30 secs; link is currently broken
[-]E1009 18:35:19.861926 602934 hyperactor/src/channel/net.rs:875] error_msg:session tcp:137.220.56.211:26600.410177472356561161: failed to deliver message within timeout
